# Examples of Visit Data from Bulter Data Products

In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
import lsst.afw.cameraGeom.testUtils as testUtils
import lsst.afw.cameraGeom as cameraGeom
from lsst.obs.hsc import HscMapper as mapper

In [2]:
REPO = '/datasets/hsc/repo/rerun/RC/w_2019_34/DM-21091'
butler = dafPersist.Butler(REPO)

In [3]:
#example dataId
dataId = {"visit": 19662, "ccd": 35}

## Seeing

In [4]:
# Seeing by Catalog:
src = butler.get('src', dataId=dataId)
radius = np.median(np.sqrt(0.5) * np.sqrt(src['slot_PsfShape_xx'] + src['slot_PsfShape_yy']))
radius

1.3509500314734233

In [5]:
miniCalexp = butler.get('calexp_sub', dataId=dataId, bbox=lsst.geom.Box2I(lsst.geom.Point2I(0,0), lsst.geom.Extent2I(1, 1)))
bbox = butler.get('calexp_bbox', dataId=dataId)

# This is the PSF at the center of the ccd which is different than in the corners:
radius = miniCalexp.getPsf().computeShape(bbox.getCenter()).getDeterminantRadius()
print(radius)

# And is not the same as the PSF in the corner:
print(miniCalexp.getPsf().computeShape(lsst.geom.Point2D(0, 0)).getDeterminantRadius())                                     

1.3342014095509867
1.3662240569626103


In [6]:
# Convert to FWHM:

SIGMA2FWHM = 2*np.sqrt(2*np.log(2))
PIXEL_SCALE = 0.168

fwhm = SIGMA2FWHM * radius
print("%.3f pixels" % fwhm)
print("%.3f arcsec" % (fwhm*PIXEL_SCALE))

3.142 pixels
0.528 arcsec


## Airmass

In [7]:
miniCalexp.getInfo().getVisitInfo().getBoresightAirmass()

1.10282413272986

## Coadd Inputs

In [8]:
coaddExp = butler.get('deepCoadd', patch="3,3", tract=9813, filter="HSC-I")
inputs = coaddExp.getInfo().getCoaddInputs()

In [9]:
inputs.visits

<class 'lsst.afw.table.exposure.exposure.ExposureCatalog'>
  id  bbox_min_x bbox_min_y bbox_max_x ... goodpix        weight       filter
         pix        pix        pix     ...                                   
----- ---------- ---------- ---------- ... -------- ------------------ ------
 1228      11900      11900      16099 ... 15712782 106.29273092461976      i
 1230      11900      11900      16099 ... 15693535 107.41940078164764      i
 1232      11900      11900      16099 ... 15687581 109.18421152345934      i
 1238      11900      11900      16099 ... 16183217   97.4457927235314      i
 1240      11900      11900      16099 ... 15691941  97.26230731179764      i
 1242      11900      11900      16099 ... 15687893  95.53641485752884      i
 1244      11900      11900      16099 ... 15712855  87.62339755344917      i
 1246      11900      11900      16099 ... 15715305  86.11495595141974      i
 1248      11900      11900      16099 ... 15712815  84.15206732339372      i
19658

In [10]:
inputs.ccds.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,ccd,visit,goodpix,weight,filter
,pix,pix,pix,pix,,,,,
int64,int32,int32,int32,int32,int32,int64,int32,float64,str32
3942016,0,0,2047,4175,16,19710,514084,80.34591582246404,i
3942017,0,0,2047,4175,17,19710,1184194,80.34591582246404,i
3942023,0,0,2047,4175,23,19710,2208215,80.34591582246404,i
3942024,0,0,2047,4175,24,19710,5614022,80.34591582246404,i
3942031,0,0,2047,4175,31,19710,1706021,80.34591582246404,i
3942032,0,0,2047,4175,32,19710,4492985,80.34591582246404,i
6096816,0,0,2047,4175,16,30484,2767394,72.35008868141928,i
6096817,0,0,2047,4175,17,30484,2478817,72.35008868141928,i


In [11]:
# You can get the inputs at a particular point too (which I'll add if you need it)

In [12]:
# Then you can loop through the rows and get each exposure:
# Note (one gotcha is that you need to make sure the visit/ccds are passed as ints)
for row in inputs.ccds:
    calexp = butler.get('calexp', visit=int(row['visit']), ccd=int(row['ccd']))
    break

In [13]:
calexp